In [1]:
import re
import importlib
import inspect
import os
import pandas as pd

In [2]:
# Function to extract C++ function definition
def extract_cpp_function_def(code_string, api_name):
    
    # Define the function pattern (simple version, assumes function returns void)
    pattern = re.compile("\w+\s+"+api_name+"\s*\([^\)]*\)\s*\{.*?\}", re.DOTALL)

    # Search for the function definition in the code string
    function_def = pattern.search(code_string)

    # If the function definition is found, return it. Otherwise, return an empty string
    return function_def.group() if function_def else ''

In [3]:
# Function to extract Python function definition
def extract_python_function_def(code_string, api_name):
    
    # Define the function pattern
    pattern = re.compile(f"def {api_name}\(.*?\):.*?(?=def |\Z)", re.DOTALL)

    # Search for the function definition in the code string
    function_def = pattern.search(code_string)

    # If the function definition is found, return it. Otherwise, return an empty string
    return function_def.group() if function_def else ''

In [4]:
#Helper fucntions
def write_to_file(file_path, content):
    with open(file_path, 'w+') as f:
        f.write(content)
        
def read_file_to_string(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
        return content
        

def is_function_defined_cpp(file_path, function_name):
    # Look for something like "void function_name(" or "int function_name("
    # This is a simplification; real C++ function definitions can be much more complex
    function_pattern = re.compile("\w+\s+"+function_name+"\s*\([^\)]*\)\s*\{.*?\}")

    with open(file_path, 'r') as file:
        content = file.read()

    return bool(function_pattern.search(content))

def is_function_defined_py(file_path, function_name):
    # Look for something like "void function_name(" or "int function_name("
    # This is a simplification; real C++ function definitions can be much more complex
    function_pattern = re.compile("def "+function_name+"\(.*?\):.*?(?=def |\Z)")

    with open(file_path, 'r') as file:
        content = file.read()

    return bool(function_pattern.search(content))

In [5]:
#Layer 1
#V3 use python funcion get source code


def get_source_code(api_name):
    module_name, class_name = api_name.rsplit('.', 1)
    
    module = importlib.import_module(module_name)
    source_code = inspect.getsource(getattr(module, class_name))

    return source_code

In [6]:
#Layer 2
#V2 search python file


def search_files_py(directory, search_string):
    module_name, search_string = search_string.rsplit('.', 1)
    for foldername, subfolders, filenames in os.walk(directory):
        for filename in filenames: 
            if filename.endswith('.py'):
                with open(os.path.join(foldername, filename), 'r') as file:
                    if search_string in file.read():
                        if is_function_defined_py(f'{foldername}/{filename}',search_string):
                            print(f'Found "{search_string}" in {foldername}/{filename}')
                            sourceCode=extract_python_function_def(read_file_to_string(f'{foldername}/{filename}'),search_string)
                            return sourceCode
    else: 
        return ''

In [7]:
#Layer 3
#V2 search cpp file

def search_files_cpp(directory, search_string):
    module_name, search_string = search_string.rsplit('.', 1)
    for foldername, subfolders, filenames in os.walk(directory):
        for filename in filenames: 
            if filename.endswith('.cpp') or filename.endswith('.h'):
                with open(os.path.join(foldername, filename), 'r') as file:
                    if search_string in file.read():
                        if is_function_defined_cpp(f'{foldername}/{filename}',search_string):
                            print(f'Found "{search_string}" in {foldername}/{filename}')
                            sourceCode=extract_cpp_function_def(read_file_to_string(f'{foldername}/{filename}'),search_string)
                            return sourceCode
    else: 
        return ''

In [14]:
def all_in_one_search(api_name):
    print("Searching on "+api_name)
    try:
        sourcecode=get_source_code(api_name)
        #print(sourcecode)
        write_to_file('SourceCode/'+api_name+'.txt',sourcecode)
        print('code recorded as'+api_name+'.txt')
        return sourcecode,1,api_name
    except:
        print('code not found when useing lib search')
        result_search_py=search_files_py('../pytorch',api_name)
        if result_search_py!='':
            print('code found in python file')
            
            return result_search_py,2,api_name
        else:
            print('searching for cpp file')
            result_search_cpp=search_files_cpp('../pytorch',api_name)
            if result_search_cpp!='':
                print('cpp code found')
                return result_search_cpp,3,api_name
            else: 
                print('nothing found return 0')
                return '',0,api_name

In [9]:

def file_avalibility_check(csv_file_path):
    # Load the data from the CSV file
    data = pd.read_csv(csv_file_path)

    # Check if 'API Name' column exists in the dataframe
    if 'API Name' in data.columns:
        data['label'] = data['API Name'].apply(all_in_one_search)!=''
        data.to_csv('FreeFuzz_torch_API_list_labeled.csv')
    else:
        print(f"'API Name' column does not exist in the CSV file: {csv_file_path}")


In [19]:
def find_source_code(csv_file_path):
    # Load the data from the CSV file
    data = pd.read_csv(csv_file_path)
    data[['Source Code', 'label', 'API Name']] = data['API Name'].apply(all_in_one_search).apply(pd.Series)
    data.to_csv('FreeFuzz_torch_API_list_code_founded.csv')

In [20]:
find_source_code('FreeFuzz_torch_API_list.csv')

Searching on torch.nn.Hardshrink
code recorded astorch.nn.Hardshrink.txt
Searching on torch.inverse
code not found when useing lib search
searching for cpp file
nothing found return 0
Searching on torch.nn.functional.max_pool2d
code recorded astorch.nn.functional.max_pool2d.txt
Searching on torch.std
code not found when useing lib search
searching for cpp file
nothing found return 0
Searching on torch.isfinite
code not found when useing lib search
searching for cpp file
nothing found return 0
Searching on torch.nn.AdaptiveMaxPool2d
code recorded astorch.nn.AdaptiveMaxPool2d.txt
Searching on torch.expm1
code not found when useing lib search
searching for cpp file
Found "expm1" in ../pytorch/aten/src/ATen/native/UnaryOps.cpp
cpp code found
Searching on torch.can_cast
code not found when useing lib search
searching for cpp file
nothing found return 0
Searching on torch.nn.NLLLoss
code recorded astorch.nn.NLLLoss.txt
Searching on torch.addmv
code not found when useing lib search
searching 

Found "rad2deg" in ../pytorch/aten/src/ATen/native/UnaryOps.cpp
cpp code found
Searching on torch.nn.functional.conv2d
code not found when useing lib search
searching for cpp file
nothing found return 0
Searching on torch.randn
code not found when useing lib search
searching for cpp file
nothing found return 0
Searching on torch.broadcast_tensors
code recorded astorch.broadcast_tensors.txt
Searching on torch.nn.ConstantPad2d
code recorded astorch.nn.ConstantPad2d.txt
Searching on torch.promote_types
code not found when useing lib search
searching for cpp file
nothing found return 0
Searching on torch.inner
code not found when useing lib search
searching for cpp file
nothing found return 0
Searching on torch.mvlgamma
code not found when useing lib search
searching for cpp file
nothing found return 0
Searching on torch.le
code not found when useing lib search
searching for cpp file
Found "le" in ../pytorch/aten/src/ATen/native/BinaryOps.cpp
cpp code found
Searching on torch.nn.Softshrink

nothing found return 0
Searching on torch.nn.MSELoss
code recorded astorch.nn.MSELoss.txt
Searching on torch.nn.InstanceNorm3d
code recorded astorch.nn.InstanceNorm3d.txt
Searching on torch.transpose
code not found when useing lib search
searching for cpp file
nothing found return 0
Searching on torch.logcumsumexp
code not found when useing lib search
searching for cpp file
nothing found return 0
Searching on torch.nn.PixelShuffle
code recorded astorch.nn.PixelShuffle.txt
Searching on torch.sign
code not found when useing lib search
searching for cpp file
Found "sign" in ../pytorch/aten/src/ATen/native/UnaryOps.cpp
cpp code found
Searching on torch.round
code not found when useing lib search
searching for cpp file
Found "round" in ../pytorch/aten/src/ATen/native/UnaryOps.cpp
cpp code found
Searching on torch.result_type
code not found when useing lib search
searching for cpp file
nothing found return 0
Searching on torch.kthvalue
code not found when useing lib search
searching for cpp 

nothing found return 0
Searching on torch.nn.MultiLabelMarginLoss
code recorded astorch.nn.MultiLabelMarginLoss.txt
Searching on torch.lstsq
code recorded astorch.lstsq.txt
Searching on torch.bincount
code not found when useing lib search
searching for cpp file
nothing found return 0
Searching on torch.digamma
code not found when useing lib search
searching for cpp file
Found "digamma" in ../pytorch/aten/src/ATen/native/UnaryOps.cpp
cpp code found
Searching on torch.nn.functional.softmax
code recorded astorch.nn.functional.softmax.txt
Searching on torch.tan
code not found when useing lib search
searching for cpp file
Found "tan" in ../pytorch/aten/src/ATen/native/UnaryOps.cpp
cpp code found
Searching on torch.nn.functional.prelu
code not found when useing lib search
searching for cpp file
nothing found return 0
Searching on torch.tanh
code not found when useing lib search
searching for cpp file
Found "tanh" in ../pytorch/aten/src/ATen/native/UnaryOps.cpp
cpp code found
Searching on tor

nothing found return 0
Searching on torch.nn.TransformerDecoderLayer
code recorded astorch.nn.TransformerDecoderLayer.txt
Searching on torch.hypot
code not found when useing lib search
searching for cpp file
nothing found return 0
Searching on torch.nn.functional.pairwise_distance
code not found when useing lib search
searching for cpp file
nothing found return 0
Searching on torch.nn.BCELoss
code recorded astorch.nn.BCELoss.txt
Searching on torch.matmul
code not found when useing lib search
searching for cpp file
nothing found return 0
Searching on torch.vander
code not found when useing lib search
searching for cpp file
nothing found return 0
Searching on torch.baddbmm
code not found when useing lib search
searching for cpp file
nothing found return 0
Searching on torch.nn.LayerNorm
code recorded astorch.nn.LayerNorm.txt
Searching on torch.symeig
code recorded astorch.symeig.txt
Searching on torch.nn.functional.relu
code recorded astorch.nn.functional.relu.txt
Searching on torch.nn.G

Found "max" in ../pytorch/c10/util/BFloat16-math.h
cpp code found
Searching on torch.sum
code not found when useing lib search
searching for cpp file
nothing found return 0
Searching on torch.nn.MaxUnpool2d
code recorded astorch.nn.MaxUnpool2d.txt
Searching on torch.angle
code not found when useing lib search
searching for cpp file
nothing found return 0
Searching on torch.nn.functional.dropout3d
code recorded astorch.nn.functional.dropout3d.txt
Searching on torch.nn.functional.softshrink
code not found when useing lib search
searching for cpp file
nothing found return 0
Searching on torch.nn.functional.conv1d
code not found when useing lib search
searching for cpp file
nothing found return 0
Searching on torch.deg2rad
code not found when useing lib search
searching for cpp file
Found "deg2rad" in ../pytorch/aten/src/ATen/native/UnaryOps.cpp
cpp code found
Searching on torch.roll
code not found when useing lib search
searching for cpp file
nothing found return 0
Searching on torch.nn.f

nothing found return 0


In [74]:
file_avalibility_check('FreeFuzz_torch_API_list.csv')

code recorded astorch.nn.Hardshrink.txt
code not found when useing lib search
code recorded astorch.nn.functional.max_pool2d.txt
code not found when useing lib search
code not found when useing lib search
code recorded astorch.nn.AdaptiveMaxPool2d.txt
code not found when useing lib search
Found "expm1" in ../pytorch/aten/src/ATen/native/UnaryOps.cpp
code not found when useing lib search
code recorded astorch.nn.NLLLoss.txt
code not found when useing lib search
code recorded astorch.nn.functional.local_response_norm.txt
code recorded astorch.norm.txt
code not found when useing lib search


KeyboardInterrupt: 

In [25]:
#Layer 3
#V2 search cpp file

def search_files_cpp_exper(directory, search_string):
    module_name, search_string = search_string.rsplit('.', 1)
    for foldername, subfolders, filenames in os.walk(directory):
        for filename in filenames: 
            if filename.endswith('.cpp') or filename.endswith('.h'):
                with open(os.path.join(foldername, filename), 'r') as file:
                    if search_string in file.read():
                        if is_function_defined_cpp(f'{foldername}/{filename}',search_string):
                            print(f'Found "{search_string}" in {foldername}/{filename}')
                            
                           


In [26]:
search_files_cpp_exper('../pytorch',"torch.inverse")

Found "inverse" in ../pytorch/caffe2/image/image_input_op.h
Found "inverse" in ../pytorch/caffe2/operators/fused_rowwise_nbitfake_conversion_ops.h
Found "inverse" in ../pytorch/caffe2/operators/fused_rowwise_nbit_conversion_ops.h
Found "inverse" in ../pytorch/caffe2/operators/quantized/int8_roi_align_op.h
Found "inverse" in ../pytorch/caffe2/sgd/learning_rate_functors.h
Found "inverse" in ../pytorch/caffe2/transforms/pattern_net_transform.h
Found "inverse" in ../pytorch/aten/src/ATen/core/aten_interned_strings.h
Found "inverse" in ../pytorch/aten/src/ATen/core/op_registration/hacky_wrapper_for_legacy_signatures.h
Found "inverse" in ../pytorch/aten/src/ATen/cpu/vec256/vec256_base.h
Found "inverse" in ../pytorch/aten/src/ATen/cpu/vec256/vec256_int.h
Found "inverse" in ../pytorch/aten/src/ATen/cpu/vec256/vec256_qint.h
Found "inverse" in ../pytorch/aten/src/ATen/cpu/vec256/vsx/vec256_qint32_vsx.h
Found "inverse" in ../pytorch/aten/src/ATen/cpu/vec256/vsx/vec256_qint8_vsx.h
Found "inverse" 